# **Риск-скоринг заемщиков** <a id='начало'></a>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель иследования:**
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Ход иследования:**
Полученны данные из data.csv, котоый хранит статистику о платёжеспособности кредитополучателей. Прежде чем ответить на вопросы поставленные Заказчиком нужно проверить качество полученных данных.

## **Обзор данных** <a id='обзор'></a>

### Импорт библиотек

In [43]:
#импортируем библиотеки 
import pandas as pd

### Загрузка данных из data.csv

In [2]:
#читаем файл и сохраняем в df
df = pd.read_csv('C://Users//sahin//Desktop//datasets//data.csv')

### **Обзор данных**

In [3]:
#выводим df
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [4]:
#выводим информацию о датафрейме
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


## **Предобработка данных** <a id='предобработка'></a>

### **Обработка пропущенных значений**

Перед обработкой пропущенный значений, посчитаем их количество по каждому столбцу.

In [5]:
#считаем пропуски в датафрейме по столбцам
print (df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В датафрейме пропущенны значения в столбце `days_employed` и `total_income`, содержащая информацию о стаже заемщика и информацию о доходах.
Посмотрим какую долю состаляют пропущенные значения в своих столбцах, в процентах.

In [6]:
#Считаем какую долю составляют пропущенные значения в каждом из столбцов с пропусками
df_income_null = len(df[df['total_income'].isna()]) / len (df)
print (f'В столбце total_income значения отсутсвуют в: {df_income_null:.1%}')

df_employed_null = len(df[df['days_employed'].isna()]) / len (df)
print (f'В столбце days_employed значения отсутсвуют в: {df_employed_null:.1%}')

В столбце total_income значения отсутсвуют в: 10.1%
В столбце days_employed значения отсутсвуют в: 10.1%


Можно предположить, что одной из причин отсувтсвия информации о доходах заемщика может быть из за начилия в Банке возможности выдачи кердита по одному документу.

In [7]:
#найдем медиану в столбце total_income и заменим пропуски в столбце на медианое значение
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)

#найдем медиану в столбце days_employed и зменим пропуски в столбце на медианое значение
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

Заполнив пропуски в столбце `total_income` и `days_emploed` мы усреднили значения, чтоюы избежать сильного искажения конечного реузльтата.
Убедимся что в столбцах не осталось пропусков.

In [8]:
print (df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### **Обработка аномалий**

#### Поиск аномалий по столбцам

Для начала посмотрим какие столбцы мы можем выделить как аномальные с помощью функции `.describe()`.

поиска аномалий будем обработывать по отедльности каждый столбец.

In [9]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Мы видим описание каждого столбца и стоить обратить внимание на следующие аномали:
- В столбце **children** мы видим значния -1 и 20;
- В столбце **days_emploed** мы видим отрицательные значения и слишком большой разрыв;
- В столбце **dob_years** мы видим нулеове значние;

Каждый столбец с аномалиями мы будем обрабатывать по отдельности.

#### Столбец **days_emploed**

Обработку значений в столбце `days_epmloyed` мы начнем с изменнения отрицательных знаков вещественных чисел.

In [10]:
#создадим функцию по замене отрицательных значений на положительные
def reverse(value):
    if value < 0:
        value *= (-1)
        return (value)
    else:
        return (value)

#заменим отрицательные значения в столбце days_employed с помощью функции reverse
df['days_employed'] = df['days_employed'].apply(reverse)

Выведим экстремальные значаения по столбце.

In [11]:
#опишем значения в столбце
df['days_employed'].describe()

count     21525.000000
mean      60277.957929
std      133301.583103
min          24.141633
25%        1025.608174
50%        1808.053434
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Теперь разберемся с причиной слишком большого разрыва в значениях max - 401755 и min - 24.
Стоит обратить внимание на то что значения в данном столбце считаются в днях, можно предполодить, что значния были записаны в часах.
Минимальный возраст для работы по трудовому договору 16 лет, а возраст для выхода на пенсию 62.5, как среднее значние.
Получается 62.5 - 16 = 46.5 лет трудового стажа или окургленно 17тыс дней (без окургления 16972 дня).
Все что больше 17тыс дней будем считать, что записанно в часах.

In [12]:
#создадим функцию, переводящая часы в дни
def employed(days):
    if days > 17000: 
        days /= 24
        return(days)
    else:
        return(days)

#обработаем столбец с помощью функции employed
df['days_employed'] = df['days_employed'].apply(employed)

Выведим экстремальные значения в столбце чтобы убедиться, что исправили аномалии.

In [13]:
df['days_employed'].describe()

count    21525.000000
mean      4292.776745
std       5181.318336
min         24.141633
25%       1025.441612
50%       1807.397693
75%       4779.202040
max      16739.808353
Name: days_employed, dtype: float64

#### Столбец **children**

Посмотрим на уникальные значения в столбце.

In [14]:
#найдем уникальные значения столбца children
print(df['children'].unique())

[ 1  0  3  2 -1  4 20  5]


Видим что в столбце есть аномальные значения -1, 20.
Предположим, что вводя -1 предполагалсь 1, а при вводе 2 было случайно 0.

In [15]:
#заменяем значения -1 на 1, в столбце children
df['children'] = df['children'].replace(-1,1)
#заменяем значения 20 на 2, в столбце children
df['children'] = df['children'].replace(20,2)
#найдем уникальные значения столбца children, чтобы убедиться, что аномальых значений нет


Выведим уникальные значения в столбце чтобы убедиться, что исправили аномалии.

In [16]:
print(df['children'].unique())

[1 0 3 2 4 5]


#### Столбец **dob_years**

Опишем значения в столбце.

In [17]:
print (df['dob_years'].describe())

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


Видим что есть значения равные 0. 
Так в РФ кредитные обязательства можно брать с 18 лет выведим значения мменьше 18. 

In [18]:
year = []
for years in df['dob_years']:
    if years < 18:
        year.append(years)
print (year)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Можно увидеть, что все значения меньше 18 равны 0. Будем считать, что нули появились в следствии отсутствия кода.
Заменим все значения равные нули на среднее значение отностильно типа занятости.

In [19]:
for type in df['income_type'].unique():
    mean_years =  df.loc[df['income_type'] == type, 'dob_years'].mean()
    df.loc[(df['income_type'] == type) & (df['dob_years'] == 0), 'dob_years'] = mean_years

C:\Users\sahin\AppData\Local\Temp\ipykernel_8528\1039318553.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.82102707077975' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df['income_type'] == type) & (df['dob_years'] == 0), 'dob_years'] = mean_years


#### Мы смогли найти несколько аномалий, обработва датафрейм мы исправили неточности в данных .  

### **Замена типа данных** 

В столбцах `total_income` и `days_employed` заменим тип данных, чтобы отбросить значения после запятой.

In [20]:
#заменим вещественный тип данных на целочисленный в столбце total_income
df['total_income'] = df['total_income'].astype(int)
#заменим вещественный тип данных на целочисленный в столбце days_employed
df['days_employed'] = df['days_employed'].astype(int)

### **Обработка дубликатов**

Проверим регистр значений в столбцах типа данных `str`.

In [21]:
#названия столбцов с типом данных str
str_columns = ['education','family_status','gender','income_type','purpose']

#выведим все уникальные значения из выбраных столбцов
for columns in str_columns:
    print (f'Уникальные значения в столбце {columns}: ')
    print(df[columns].unique())
    print()

Уникальные значения в столбце education: 
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Уникальные значения в столбце family_status: 
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения в столбце gender: 
['F' 'M' 'XNA']

Уникальные значения в столбце income_type: 
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Уникальные значения в столбце purpose: 
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'н

Видно что в неокторых столбцах значения записаны разным регистром.     
Приведем все значения к одному регистру с пмощью функции `.str.lower()`.

In [22]:
#в столбцах с типом данных str, приведем все значения к одному регистру
for columns_name in str_columns:
    df[columns_name] = df[columns_name].str.lower()

Теперь найдем дубликаты с помощью функции `.duplicated()` и посчитаем их количество.

In [23]:
#найдем дубликаты    
print (f'Найдено дубликатов: {df.duplicated().sum()}')

Найдено дубликатов: 71


Удалим все обнаруженные дубликаты с помощью функции `.drop_duplicates()`.

In [24]:
#удаляем все дубликаты
df = df.drop_duplicates()

Убедимся что удалили все дубликаты.

In [44]:
print (f'Найдено дубликатов: {df.duplicated().sum()}.')

Найдено дубликатов: 0.


## **Создание новых датафреймов**

### Создание **df_education**

Создадим `df_education` на основе столбцов `education_id` и `education` из `df`.

In [26]:
#создаем новый даатафрейм хранящий в себе столбцы education_id и education из df
df_education = df[['education_id','education']]

Выведем первые 10 строк `df_education`.

In [27]:
print(df_education.head(10))

   education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
5             0    высшее
6             0    высшее
7             1   среднее
8             0    высшее
9             1   среднее


Удалим столбец `education` из `df`.

In [28]:
del df['education']

### Создание **df_family_status**

Создадим `df_family_status` на основе столбцов `family_status_id` и `family_status` из `df`.

In [29]:
#создаем новый даатафрейм хранящий в себе столбцы family_status_id и family_status из df
df_family_status = df[['family_status_id','family_status']]

Выведем первые 10 строк `df_family_status`.

In [30]:
print(df_family_status.head(10))

   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем


Удалим столбец `family_status` из `df`.

In [31]:
del df['family_status']

## **Категоризация**

#### Создание столбца **total_income_category**

Категоризируем доходы кредитополучателей на основании диапазонов, указанных ниже:
- 0–30000 — **E**;
- 30001–50000 — **D**;
- 50001–200000 — **C**;
- 200001–1000000 — **B**;
- 1000001 и выше — **A**;

Создадим функцию `categorization_total_income` для категоризации значений.

In [32]:
#создадим функцию для категоризации значений
def categorization_total_income(category):
    try:
        if 30000 > category > 0:
            return('E')
        elif 50000 > category > 30001:
            return('D')
        elif 200000 > category > 50001:
            return('C')
        elif 1000000 > category > 200001:
            return('B')
        elif category > 1000001:
            return('A')
    except:
        return('нет категории')

Теперь сформируем столбец `total_income_category`,который категоризирует значения в столбце `total_income`.

In [33]:
#обработаем столбец с помощью функции categorization_total_income, 
#c сохранением в новом столбце total_income_category
df['total_income_category'] = df['total_income'].apply(categorization_total_income)

C:\Users\sahin\AppData\Local\Temp\ipykernel_8528\559591598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_income_category'] = df['total_income'].apply(categorization_total_income)


Посмторим как выглядит датафрейм с новым столбцом `total_income_category`. 

In [34]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,f,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,m,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,14177,53.0,1,1,f,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27.0,0,1,m,компаньон,0,255763,покупка жилья,B
6,0,2879,43.0,0,0,f,компаньон,0,240525,операции с жильем,B
7,0,152,50.0,1,0,m,сотрудник,0,135823,образование,C
8,2,6929,35.0,0,1,f,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41.0,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C


#### Создание столбца **purpose_category**

Выведим цели получения кредита, чтобы выделить основыне категории.

In [35]:
print ('Цели получения кредита: ', df['purpose'].unique())

Цели получения кредита:  ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Из целей получения кредита мы можем выделить следующие категории:
- Операции с автомобилем;
- Операции с недвижимостью;
- Получение образования;
- Проведение свадьбы;

Создадим функцию `categorization_purpose` для категоризации значений.

In [36]:
#создадим функцию для категоризации значений
def categorization_purpose(category):
    try:
        if 'образов' in category:
            return('получение образования')
        elif 'авто' in category:
            return('операции с автомобилем')
        elif ('жил' in category) or ('недвиж' in category):
            return('операции с недвижимостью')
        elif 'свад' in category:
            return('проведение свадьбы')
    except:
        return('нет категории')

Теперь сформируем новый столбец `purpose_category`,который категоризирует значения в столбце `purpose`.

In [37]:
#обработаем столбец с помощью функции categorization, 
#c сохранением в новом столбце purpose_category
df['purpose_category'] = df['purpose'].apply(categorization_purpose)

C:\Users\sahin\AppData\Local\Temp\ipykernel_8528\1666098698.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['purpose_category'] = df['purpose'].apply(categorization_purpose)


Посмторим как выглядит датафрейм с новым столбцом `purpose_category`. 

In [38]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42.0,0,0,f,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,1,0,f,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,1,0,m,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,1,0,m,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177,53.0,1,1,f,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27.0,0,1,m,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43.0,0,0,f,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50.0,1,0,m,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35.0,0,1,f,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41.0,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## **Ответы на вопросы**

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на данный вопрос создадим таблицу c процентным соотношением.

In [39]:
#создадим датафрейм
children_debt = pd.DataFrame()
#посчитаем отношение в процентах
children_debt['debt_sum'] = df.groupby('children')['debt'].sum()
children_debt['debt_count'] = df.groupby('children')['debt'].count()
children_debt['debt'] = round((children_debt['debt_sum'] / children_debt['debt_count']) * 100, 2)
children_debt.sort_values(by='debt', ascending = False)

,debt_sum,debt_count,debt
children,,,
4,4,41,9.76
2,202,2128,9.49
1,445,4855,9.17
3,27,330,8.18
0,1063,14091,7.54
5,0,9,0.00


Из сформированной таблицы мы видим, что бездетные реже просрачивают оплату по кредиту. С ростом количества детей видно, что растет и вероятность просрочки оплаты по кредиту.


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на данный вопрос создадим таблицу c процентным соотношением.

In [40]:
#создадим датафрейм
family_status_debt = pd.DataFrame()

#добавим к df_family_status столбец 'debt'
df_family_status['debt'] = df['debt']

#посчитаем отношение в процентах
family_status_debt['debt_sum'] = df_family_status.groupby('family_status')['debt'].sum()
family_status_debt['debt_count'] = df_family_status.groupby('family_status')['debt'].count()
family_status_debt['debt'] = round((family_status_debt['debt_sum'] / family_status_debt['debt_count']) * 100, 2)
family_status_debt.sort_values(by='debt', ascending = False)

C:\Users\sahin\AppData\Local\Temp\ipykernel_8528\2096640794.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_family_status['debt'] = df['debt']


,debt_sum,debt_count,debt
family_status,,,
не женат / не замужем,274,2810,9.75
гражданский брак,388,4151,9.35
женат / замужем,931,12339,7.55
в разводе,85,1195,7.11
вдовец / вдова,63,959,6.57


Из сформированной таблицы мы видим, что не женатые/не замужние просрачивают оплату по кредиту чаще, нежели женатые/замужние. Но, те кто состоял в браке чаще опалчивают кредит в срок, чем состоящие в браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на данный вопрос создадим таблицу c процентным соотношением.

In [41]:
#создадим датафрейм
total_income_debt = pd.DataFrame()
#посчитаем отношение в процентах
total_income_debt['debt_sum'] = df.groupby('total_income_category')['debt'].sum()
total_income_debt['debt_count'] = df.groupby('total_income_category')['debt'].count()
total_income_debt['debt'] = round((total_income_debt['debt_sum'] / total_income_debt['debt_count']) * 100, 2)
total_income_debt 

,debt_sum,debt_count,debt
total_income_category,,,
A,2,25,8.00
B,356,5040,7.06
C,1360,16016,8.49
D,21,350,6.00
E,2,22,9.09


### Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос создадим таблицу c процентным соотношением.

In [42]:
#создадим датафрейм
purpose_debt = pd.DataFrame()
#посчитаем отношение в процентах
purpose_debt['debt_sum'] = df.groupby('purpose_category')['debt'].sum()
purpose_debt['debt_count'] = df.groupby('purpose_category')['debt'].count()
purpose_debt['debt'] = round((purpose_debt['debt_sum'] / purpose_debt['debt_count']) * 100, 2)
purpose_debt.sort_values(by='debt', ascending = False)

,debt_sum,debt_count,debt
purpose_category,,,
операции с автомобилем,403,4306,9.36
получение образования,370,4013,9.22
проведение свадьбы,186,2324,8.00
операции с недвижимостью,782,10811,7.23


Из сформированной нами таблицы мы видим, что чаще всего просрачивают опалту по кредиту на операции с автомобилем,  
реже всего просрачивают опалту по кредиту на операции с недвижимостью.

## **Общий вывод**

На основе проанализрованных нами данных можно сделать вывод, что существует зависимость между семейным положением и колличеством детей на факт погашения кредита, и зависимость между уровнем дохода и возвратом кредита в срок. 
Также можно выделить уровни дохода и категории трат кредита, которые можно выделить как группу риска. 